# Using tf.data and in-memory BigQuery DataFrames

This notebook will show you how to extract a in-memory sized training set from BigQuery to a Pandas DataFrame, then feed it into a TensorFlow model with the tf.data APi.

Copyright 2021 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [1]:
# examine the dataset from the CLI
!bq head -n 5 bigquery-public-data:london_bicycles.cycle_hire

+-----------+----------+---------+---------------------+----------------+-------------------------------+---------------------+------------------+---------------------------------------------+------------------------------+--------------------------------+-------------------------+
| rental_id | duration | bike_id |      end_date       | end_station_id |       end_station_name        |     start_date      | start_station_id |             start_station_name              | end_station_logical_terminal | start_station_logical_terminal | end_station_priority_id |
+-----------+----------+---------+---------------------+----------------+-------------------------------+---------------------+------------------+---------------------------------------------+------------------------------+--------------------------------+-------------------------+
|  47469109 |     3180 |    7054 | 2015-09-03 12:45:00 |            111 | Park Lane , Hyde Park         | 2015-09-03 11:52:00 |              300 | Serp

In [2]:
# !pip install google-cloud-bigquery-storage

In [3]:
from google.cloud import bigquery

In [4]:
%%bigquery df_from_magic --use_bqstorage_api
SELECT * FROM 
bigquery-public-data.london_bicycles.cycle_hire
WHERE EXTRACT(YEAR from start_date) = 2017
AND EXTRACT(MONTH from start_date) = 1


In [5]:
df_from_magic.head()

,rental_id,duration,bike_id,end_date,end_station_id,end_station_name,start_date,start_station_id,start_station_name,end_station_logical_terminal,start_station_logical_terminal,end_station_priority_id
0,61665038,1920,4698,2017-01-13 17:21:00+00:00,766,"Ram Street, Wandsworth",2017-01-13 16:49:00+00:00,815,"Lambeth Palace Road, Waterloo",NaN,NaN,NaN
1,61412525,1920,9696,2017-01-02 17:22:00+00:00,485,"Old Ford Road, Bethnal Green",2017-01-02 16:50:00+00:00,480,"Flamborough Street, Limehouse",NaN,NaN,NaN
2,61656964,1920,9031,2017-01-13 09:26:00+00:00,795,"Melton Street, Euston",2017-01-13 08:54:00+00:00,698,"Shoreditch Court, Haggerston",NaN,NaN,NaN
3,61556031,1920,2613,2017-01-09 15:13:00+00:00,113,"Gloucester Road (Central), South Kensington",2017-01-09 14:41:00+00:00,15,"Great Russell Street, Bloomsbury",NaN,NaN,NaN
4,61689012,1920,9814,2017-01-15 10:54:00+00:00,596,"Parson's Green , Parson's Green",2017-01-15 10:22:00+00:00,596,"Parson's Green , Parson's Green",NaN,NaN,NaN


In [6]:
client = bigquery.Client()

In [7]:
query_string = """
SELECT duration, start_station_id,
  EXTRACT(DAYOFWEEK from start_date) as day_of_week,
  EXTRACT(HOUR from start_date) as hour

FROM 
bigquery-public-data.london_bicycles.cycle_hire
WHERE EXTRACT(YEAR from start_date) = 2017
AND EXTRACT(MONTH from start_date) = 1
"""

In [8]:
df = client.query(query_string).to_dataframe()

In [9]:
df.head()

,duration,start_station_id,day_of_week,hour
0,36720,401,7,5
1,4920,99,1,3
2,2160,419,1,5
3,2100,453,1,4
4,2040,541,1,3


In [10]:
#TODO(dhodun): add categorial support, station id is not real continuous feature

In [11]:
import pandas as pd
import tensorflow as tf

In [12]:
df.dtypes

duration            int64
start_station_id    int64
day_of_week         int64
hour                int64
dtype: object

In [13]:
target = df.pop('duration')

In [14]:
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))

In [15]:
for feat, targ in dataset.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: [401   7   5], Target: 36720
Features: [99  1  3], Target: 4920
Features: [419   1   5], Target: 2160
Features: [453   1   4], Target: 2100
Features: [541   1   3], Target: 2040


In [16]:
train_dataset = dataset.shuffle(len(df)).batch(64).prefetch(1)
# 1=AUTOTUNE

In [17]:
from tensorflow import keras

In [18]:
# Simple model shown for simplicity and using the tf.data API

model = keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [19]:
model.compile(optimizer='adam', loss='mean_absolute_error')

In [ ]:
model.fit(train_dataset, epochs=2)

Train for 9882 steps
Epoch 1/2
3144/9882 [========>.....................] - ETA: 47s - loss: 634.0907

Copyright 2020 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.